In [2]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os
from io import StringIO
from Tools import *

In [3]:
#讀取INI檔案
InitFile = IniTool()

Load init file: .\Initialization.ini


In [4]:
#轉換日期民國到西元
def date_convert(strDate):
    arydate = strDate.split("/")
    return date(int(arydate[0]) + 1911, int(arydate[1]), int(arydate[2])).strftime('%Y/%m/%d')

In [5]:
# 爬取每月股價的目標網站並包裝成函式
def craw_one_month(stock_number,date):
    url = (
        "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
        date.strftime('%Y%m%d')+
        "&stockNo="+
        str(stock_number)
    )
    data = json.loads(urlopen(url).read())
    return pd.DataFrame(data['data'],columns=data['fields'])

In [6]:
# 根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
def craw_stock(stock_number, start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = dtime.now().strftime("%Y-%m-%d")  # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    
    result = pd.DataFrame()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        print(dt)
        result = pd.concat([result, craw_one_month(stock_number, dt)], ignore_index=True)
        time.sleep(10000.0 / 1000.0);

    return result

In [68]:
def ConvertType_NewData(df):
    #轉換日期民國到西元.
    df['日期'] = [date_convert(x) for x in df['日期']]
    df['日期'] = pd.to_datetime(df['日期'].astype(str), format='%Y/%m/%d')
    #轉換型態.
    if df['成交筆數'].dtype != 'float64':
        df['成交筆數'] = df['成交筆數'].str.replace(',','').astype(float)
    if df['成交股數'].dtype != 'float64':
        df['成交股數'] = df['成交股數'].str.replace(',','').astype(float)
    if df['成交金額'].dtype != 'float64':
        df['成交金額'] = df['成交金額'].str.replace(',','').astype(float)

In [7]:
# 建立資料夾
def MKDir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [12]:
NoHistory = False

In [13]:
DataPath = InitFile.Read('Infomation', 'DataPath')
Code = InitFile.Read('Infomation', 'StockCode')
Date = InitFile.Read('Infomation', 'StartDate')
print(Code)
print(Date)
print(DataPath)
fullpath = DataPath + Code + '.csv'
print(fullpath) 

2303
2020-01-01
./Data/
./Data/2303.csv


In [14]:
#確認是否有舊資料
if os.path.isfile(fullpath):
    #檔案存在的處理
    print("檔案存在。")
else:
    #檔案不存在的處理
    print("檔案不存在。")
    NoHistory = True

檔案不存在。


In [15]:
df_History = pd.DataFrame()

In [16]:
if NoHistory is True:
    df_History = craw_stock(Code,Date)

2020-01-01 00:00:00
2020-02-01 00:00:00
2020-03-01 00:00:00
2020-04-01 00:00:00
2020-05-01 00:00:00
2020-06-01 00:00:00
2020-07-01 00:00:00
2020-08-01 00:00:00
2020-09-01 00:00:00
2020-10-01 00:00:00
2020-11-01 00:00:00
2020-12-01 00:00:00


In [17]:
#測試用變數
df_His = df_History.copy()

In [18]:
if NoHistory is True:
    #轉換日期民國到西元.
    df_His['日期'] = [date_convert(x) for x in df_His['日期']]
    df_His['日期'] = pd.to_datetime(df_His['日期'].astype(str), format='%Y/%m/%d')
    #轉換型態.
    if df_His['成交筆數'].dtype != 'float64':
        df_His['成交筆數'] = df_His['成交筆數'].str.replace(',','').astype(float)
    if df_His['成交股數'].dtype != 'float64':
        df_His['成交股數'] = df_His['成交股數'].str.replace(',','').astype(float)
    if df_His['成交金額'].dtype != 'float64':
        df_His['成交金額'] = df_His['成交金額'].str.replace(',','').astype(float)

In [21]:
df_His.to_csv(fullpath, encoding='utf_8_sig')

In [22]:
df_His.dtypes

日期      datetime64[ns]
成交股數           float64
成交金額           float64
開盤價             object
最高價             object
最低價             object
收盤價             object
漲跌價差            object
成交筆數           float64
dtype: object

In [23]:
#清空
df_His = pd.DataFrame()

In [155]:
df_His = pd.read_csv(fullpath, index_col = 0)
df_His

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2020/1/2,32937461,544863192,16.45,16.65,16.40,16.55,0.1,5382
1,2020/1/3,48588354,792990409,16.55,16.55,16.15,16.30,-0.25,9541
2,2020/1/6,44790472,719002202,16.05,16.15,15.95,16.05,-0.25,5041
3,2020/1/7,39718117,633334895,16.05,16.15,15.80,16.00,-0.05,7401
4,2020/1/8,60494690,954618728,15.85,15.90,15.65,15.75,-0.25,9368
...,...,...,...,...,...,...,...,...,...
228,2020/12/9,321064214,15950783936,49.10,50.20,49.10,49.90,-1,105914
229,2020/12/10,460980326,22003933802,48.60,48.90,46.70,47.30,-2.6,162363
230,2020/12/11,390831102,18150209771,47.45,48.35,45.20,46.10,-1.2,120021
231,2020/12/14,233249964,10680175679,46.10,46.85,44.60,46.15,0.05,74436


In [156]:
df_His.dtypes

日期       object
成交股數      int64
成交金額      int64
開盤價     float64
最高價     float64
最低價     float64
收盤價     float64
漲跌價差     object
成交筆數      int64
dtype: object

In [157]:
#轉換日期格式.
df_His['日期'] = pd.to_datetime(df_His['日期'].astype(str), format='%Y/%m/%d')

In [158]:
df_His.dtypes

日期      datetime64[ns]
成交股數             int64
成交金額             int64
開盤價            float64
最高價            float64
最低價            float64
收盤價            float64
漲跌價差            object
成交筆數             int64
dtype: object

In [159]:
#取得今天日期
today = date.today()

In [160]:
#取的今天的表格
df_Today = craw_one_month(Code,today)
# type(Code)
# str(Code)

In [161]:
df_Today.dtypes

日期      object
成交股數    object
成交金額    object
開盤價     object
最高價     object
最低價     object
收盤價     object
漲跌價差    object
成交筆數    object
dtype: object

In [162]:
ConvertType_NewData(df_Today)

In [163]:
#找出歷史的最後一天和目前的最後一天進行比較
lastday_his = df_His.loc[:,'日期'].max()
lastday_his

Timestamp('2020-12-15 00:00:00')

In [164]:
lastday_today = df_Today.loc[:,'日期'].max()
lastday_today

Timestamp('2020-12-18 00:00:00')

In [165]:
between = lastday_today - lastday_his

In [166]:
between.days

3

In [167]:
lastday_his2 = lastday_his - timedelta(days=lastday_his.day-1)
lastday_his2

Timestamp('2020-12-01 00:00:00')

In [168]:
df_Today.loc[df_Today['日期'] > lastday_his]

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
11,2020-12-16,250325714.0,1.133766e+10,44.60,46.10,44.60,44.90,+0.90,78525.0
12,2020-12-17,189154904.0,8.634708e+09,45.35,46.40,44.90,46.05,+1.15,57672.0
13,2020-12-18,230929009.0,1.075046e+10,47.00,47.40,45.80,45.85,-0.20,64945.0


In [169]:
df_His

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2020-01-02,32937461,544863192,16.45,16.65,16.40,16.55,0.1,5382
1,2020-01-03,48588354,792990409,16.55,16.55,16.15,16.30,-0.25,9541
2,2020-01-06,44790472,719002202,16.05,16.15,15.95,16.05,-0.25,5041
3,2020-01-07,39718117,633334895,16.05,16.15,15.80,16.00,-0.05,7401
4,2020-01-08,60494690,954618728,15.85,15.90,15.65,15.75,-0.25,9368
...,...,...,...,...,...,...,...,...,...
228,2020-12-09,321064214,15950783936,49.10,50.20,49.10,49.90,-1,105914
229,2020-12-10,460980326,22003933802,48.60,48.90,46.70,47.30,-2.6,162363
230,2020-12-11,390831102,18150209771,47.45,48.35,45.20,46.10,-1.2,120021
231,2020-12-14,233249964,10680175679,46.10,46.85,44.60,46.15,0.05,74436


In [170]:
#合併資料
result = pd.DataFrame()
result = pd.concat([result, df_His], ignore_index=True)
result = pd.concat([result, df_Today.loc[df_Today['日期'] > lastday_his]], ignore_index=True)
result

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2020-01-02,32937461.0,5.448632e+08,16.45,16.65,16.4,16.55,0.1,5382.0
1,2020-01-03,48588354.0,7.929904e+08,16.55,16.55,16.15,16.3,-0.25,9541.0
2,2020-01-06,44790472.0,7.190022e+08,16.05,16.15,15.95,16.05,-0.25,5041.0
3,2020-01-07,39718117.0,6.333349e+08,16.05,16.15,15.8,16,-0.05,7401.0
4,2020-01-08,60494690.0,9.546187e+08,15.85,15.9,15.65,15.75,-0.25,9368.0
...,...,...,...,...,...,...,...,...,...
231,2020-12-14,233249964.0,1.068018e+10,46.1,46.85,44.6,46.15,0.05,74436.0
232,2020-12-15,275779367.0,1.239335e+10,46.1,46.55,44,44,-2.15,82136.0
233,2020-12-16,250325714.0,1.133766e+10,44.60,46.10,44.60,44.90,+0.90,78525.0
234,2020-12-17,189154904.0,8.634708e+09,45.35,46.40,44.90,46.05,+1.15,57672.0


In [ ]:
df

In [ ]:
#複製測試用
df2 = df.copy()
df2

In [ ]:
df2['日期'] = pd.to_datetime(df2['日期'].astype(str), format='%Y/%m/%d')

In [ ]:
#轉換型態.
if df2['成交筆數'].dtype != 'float64':
    df2['成交筆數'] = df2['成交筆數'].str.replace(',','').astype(float)
if df2['成交股數'].dtype != 'float64':
    df2['成交股數'] = df2['成交股數'].str.replace(',','').astype(float)
if df2['成交金額'].dtype != 'float64':
    df2['成交金額'] = df2['成交金額'].str.replace(',','').astype(float)

In [ ]:
df2
df2.dtypes

In [ ]:
df2.loc[:,'日期'].max()

In [ ]:
df2.loc[df2["日期"]==df2.loc[:,"日期"].max(),"日期"]

In [ ]:
#讀取原有的資料.
df_History = pd.read_csv(fullpath)

In [ ]:
df_History.dtypes
df_History

In [ ]:
#轉換型態.
df_History['日期'] = pd.to_datetime(df_History['日期'].astype(str), format='%Y/%m/%d')
if df_History['成交筆數'].dtype != 'float64':
    df_History['成交筆數'] = df_History['成交筆數'].str.replace(',','').astype(float)
if df_History['成交股數'].dtype != 'float64':
    df_History['成交股數'] = df_History['成交股數'].str.replace(',','').astype(float)
if df_History['成交金額'].dtype != 'float64':
    df_History['成交金額'] = df_History['成交金額'].str.replace(',','').astype(float)